In [ ]:
#import system packages
import os
import sys

#import langchain modules
from langchain_ollama import ChatOllama
from langchain_core.documents import Document
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import (unstructured,
                                                  UnstructuredExcelLoader,
                                                  CSVLoader,
                                                  PyMuPDFLoader,
                                                  Docx2txtLoader,
                                     dd             )
from langchain import hub
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import ChatPromptTemplate

In [ ]:
llm = ChatOllama(
   model="llama3",
   temperature=0,)
prompt = ChatPromptTemplate.from_template("Tell me a joke about {topic}")
chain = prompt | llm | StrOutputParser()
for chunk in chain.stream({"topic": "McDonald's"}):
   print(chunk, end = "", flush = True)


In [ ]:
prompt = hub.pull("rlm/rag-prompt")
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

rag_chain.invoke("What is Task Decomposition?")